In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Port Elizabeth,-33.9180,25.5701,61.11,81,0,9.22,ZA,2022-06-05 18:14:48
1,1,Batouri,4.4333,14.3667,69.42,89,90,2.73,CM,2022-06-05 18:14:49
2,2,Tilichiki,60.4667,166.1000,42.67,91,51,5.73,RU,2022-06-05 18:14:49
3,3,Ushuaia,-54.8000,-68.3000,47.86,37,75,9.22,AR,2022-06-05 18:12:57
4,4,Manto,14.9167,-86.3833,78.98,71,100,1.57,HN,2022-06-05 18:14:49


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date           object
dtype: object

In [4]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
# Heatmap of temperature
# Get the latitude and longitude.
locations = city_data_df[["Lat", "Lng"]]
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
# Assign the figure variable.
fig = gmaps.figure()
# Assign the heatmap variable.
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)
# Add the heatmap layer.
fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [9]:
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp)


In [10]:
# Get the maximum temperature.
max_temp = city_data_df["Max Temp"]
temps = []
for temp in max_temp:
    temps.append(max(temp, 0))

In [11]:
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)


In [12]:
# Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [15]:
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Manto,14.9167,-86.3833,78.98,71,100,1.57,HN,2022-06-05 18:14:49
5,5,Hilo,19.7297,-155.0900,79.29,92,100,5.75,US,2022-06-05 18:14:50
6,6,Naze,28.3667,129.4833,76.53,86,100,22.19,JP,2022-06-05 18:14:51
7,7,Akyab,20.1500,92.9000,81.52,82,83,4.47,MM,2022-06-05 18:14:51
16,16,Port-Gentil,-0.7193,8.7815,76.96,88,40,6.91,GA,2022-06-05 18:14:55
18,18,Puerto Ayora,-0.7393,-90.3518,78.76,92,96,8.99,EC,2022-06-05 18:14:55
21,21,Atuona,-9.8000,-139.0333,78.13,72,36,16.40,PF,2022-06-05 18:14:57
23,23,Tual,-5.6667,132.7500,82.04,71,96,11.23,ID,2022-06-05 18:14:57
25,25,Luganville,-15.5333,167.1667,79.56,80,63,10.40,VU,2022-06-05 18:14:58
29,29,Praia,14.9215,-23.5087,81.14,61,0,23.02,CV,2022-06-05 18:14:59


In [16]:
preferred_cities_df.count()


City_ID       186
City          186
Lat           186
Lng           186
Max Temp      186
Humidity      186
Cloudiness    186
Wind Speed    186
Country       185
Date          186
dtype: int64

In [17]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Manto,HN,78.98,14.9167,-86.3833,
5,Hilo,US,79.29,19.7297,-155.0900,
6,Naze,JP,76.53,28.3667,129.4833,
7,Akyab,MM,81.52,20.1500,92.9000,
16,Port-Gentil,GA,76.96,-0.7193,8.7815,
18,Puerto Ayora,EC,78.76,-0.7393,-90.3518,
21,Atuona,PF,78.13,-9.8000,-139.0333,
23,Tual,ID,82.04,-5.6667,132.7500,
25,Luganville,VU,79.56,-15.5333,167.1667,
29,Praia,CV,81.14,14.9215,-23.5087,


In [18]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [26]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [27]:
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
4,Manto,HN,78.98,14.9167,-86.3833,
5,Hilo,US,79.29,19.7297,-155.0900,Hilo Hawaiian Hotel
6,Naze,JP,76.53,28.3667,129.4833,Shishime Hotel
7,Akyab,MM,81.52,20.1500,92.9000,Hotel Memory
16,Port-Gentil,GA,76.96,-0.7193,8.7815,Fengshui Residence
18,Puerto Ayora,EC,78.76,-0.7393,-90.3518,Finch Bay Galapagos Hotel
21,Atuona,PF,78.13,-9.8000,-139.0333,Villa Enata
23,Tual,ID,82.04,-5.6667,132.7500,Grand Vilia Hotel
25,Luganville,VU,79.56,-15.5333,167.1667,The Espiritu Hotel
29,Praia,CV,81.14,14.9215,-23.5087,LT Aparthotel


In [28]:
# Add a heatmap of temperature for the vacation spots.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [29]:
# Add a heatmap of temperature for the vacation spots and marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [30]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{column1}</dd>
<dt>Another name</dt><dd>{column2}</dd>
</dl>
"""

In [31]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [32]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [33]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))